In [1]:
%matplotlib inline  

import sys, os, time, math
import cPickle as pickle
import tensorflow as tf
sys.path.append('/Users/matt.meng/dev/seq2seq_model')
from data import DataGenerator, process_batch
from utils import model_meta_file
from data_preprocess import TOKEN_DICT, _GO, _EOS
import matplotlib.pyplot as plt

In [2]:
#pickle_file = 'small_processed_titles_data.pkl'
#local_model_path = '/Users/matt.meng/local_tensorflow_content/seq2seq'
#local_model_path = '/Users/matt.meng/local_tensorflow_content/seq2seq_64embed_128hid'
pickle_file = 'processed_titles_data.pkl'
#local_model_path = '/Users/matt.meng/local_tensorflow_content/large_32embed_256hid'
local_model_path = '/Users/matt.meng/local_tensorflow_content/large_32embed_64hid'

In [3]:
#with open(pickle_file_path, 'rb') as input_stream:
#    data = pickle.load(input_stream)

In [4]:
encoder_output_c_name = 'encoder_decoder_sequence/encoder/while/Exit_2:0'
encoder_output_h_name = 'encoder_decoder_sequence/encoder/while/Exit_3:0'


pickle_file_path = os.path.join(os.path.expanduser("~"), pickle_file)
with open(pickle_file_path, 'rb') as input_stream:
    data = pickle.load(input_stream)
titles = data['titles']
reverse_token_dict = data['reverse_token_dict']
title_urls = data['url']
title_pageViews = data['pageViw']

In [5]:
data.keys()

['url', 'reverse_token_dict', 'pageViw', 'titles', 'token_dict']

In [6]:
#c_vector_list, h_vector_list = [], []
vector_dict = {}
saver = tf.train.import_meta_graph(model_meta_file(local_model_path))
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint(local_model_path))
    
    encoder_inputs = sess.graph.get_tensor_by_name("initial_inputs/encoder_inputs:0")
    decoder_inputs = sess.graph.get_tensor_by_name("initial_inputs/decoder_inputs:0")
    decoder_targets = sess.graph.get_tensor_by_name("initial_inputs/decoder_targets:0")
    dropout_input_keep_prob = sess.graph.get_tensor_by_name("initial_inputs/dropout_input_keep_prob:0")

    encoder_output_c = sess.graph.get_tensor_by_name('encoder_decoder_sequence/encoder/while/Exit_2:0')
    encoder_output_h = sess.graph.get_tensor_by_name('encoder_decoder_sequence/encoder/while/Exit_3:0')
    counter = 0
    start_time = time.time()
    for i in xrange(len(titles)):
        counter += 1
        vector_dict[title_urls[i]] = {'pageView' : title_pageViews[i], 
                                      'title_string' : ' '.join(map(reverse_token_dict.get, titles[i]))}
        #title = ' '.join(map(reverse_token_dict.get, titles[i]))
        encoder_inputs_, _ = process_batch([titles[i] + [TOKEN_DICT[_EOS]]])
        decoder_targets_, _ = process_batch([titles[i] + [TOKEN_DICT[_EOS]]])
        decoder_inputs_, _ = process_batch([[TOKEN_DICT[_GO]] + titles[i]])
        c_vector, h_vector = sess.run([encoder_output_c, encoder_output_h], {encoder_inputs : encoder_inputs_,
                                                                             decoder_inputs : decoder_inputs_, 
                                                                             decoder_targets : decoder_targets_, 
                                                                             dropout_input_keep_prob : 1.})
        vector_dict[title_urls[i]]['vectors'] = (c_vector[0] + h_vector[0])
        #c_vector_list.append(c_vector)
        #h_vector_list.append(h_vector)
        if counter % 5000 == 0:
            print 'finish processing {} articles using {:.2f} seconds...'.format(counter, time.time() - start_time)

warning, more than one model meta file is found in /Users/matt.meng/local_tensorflow_content/large_32embed_64hid
INFO:tensorflow:Restoring parameters from /Users/matt.meng/local_tensorflow_content/large_32embed_64hid/models-1650000
finish processing 5000 articles using 8.30 seconds...
finish processing 10000 articles using 16.57 seconds...
finish processing 15000 articles using 25.58 seconds...
finish processing 20000 articles using 34.59 seconds...
finish processing 25000 articles using 43.91 seconds...
finish processing 30000 articles using 53.76 seconds...
finish processing 35000 articles using 63.92 seconds...
finish processing 40000 articles using 73.52 seconds...
finish processing 45000 articles using 83.56 seconds...
finish processing 50000 articles using 94.37 seconds...
finish processing 55000 articles using 105.54 seconds...
finish processing 60000 articles using 117.03 seconds...
finish processing 65000 articles using 128.47 seconds...
finish processing 70000 articles using 

In [ ]:
# the difference is caused by duplicate URLs
print len(title_pageViews), len(vector_dict)

In [ ]:
key = vector_dict.keys()[10]
print vector_dict[key].keys()

In [ ]:
vector_items = vector_dict.items()

In [ ]:
sorted_vecor = sorted(vector_items, key=lambda x: x[1]['pageView'], reverse=True)

In [ ]:
sorted_vecor[10]

In [ ]:
#vector_dict['http://www.chicagotribune.com/lifestyles/sns-201706141112--tms--hscopebctnzz-a20170708-20170708-story.html']

In [ ]:
def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))
 
def euclidean_distance(x,y):
     return math.sqrt(sum(math.pow(a - b, 2) for a, b in zip(x, y)))

def manhattan_distance(x,y): 
    return sum(abs(a - b) for a,b in zip(x,y))

def cosine_similarity(x, y):
    numerator = sum(a*b for a, b in zip(x,y))
    denominator = square_rooted(x) * square_rooted(y)
    #denominator = (square_rooted(x) + square_rooted(y))

    return numerator / denominator

In [ ]:
index = 230
print sorted_vecor[index][1]['title_string']
selected_key = sorted_vecor[index][0]
print selected_key

#### use `index` to loop through all the articles

In [ ]:
cosine_results, euclidean_results = [], []
#selected_key = 'http://www.cbssports.com/nba/news/nba-draft-lavar-ball-guarantees-lonzo-will-lead-lakers-to-playoffs-next-season/'
print selected_key
print '\n \n'
start_time = time.time()
#for i in xrange(len(sorted_vecor)):
for i in xrange(100000):
    if i == index:
        continue
    cosine_result = cosine_similarity(sorted_vecor[index][1]['vectors'], sorted_vecor[i][1]['vectors'])
    euclidean_result = euclidean_distance(sorted_vecor[index][1]['vectors'], sorted_vecor[i][1]['vectors'])
    cosine_results.append(cosine_result)
    euclidean_results.append(euclidean_result)
print 'all the process takes {:.2f} seconds...'.format(time.time() - start_time)

In [ ]:
#plt.hist(euclidean_results, bins=50)
plt.hist(cosine_results, bins=50)

In [ ]:
#cosine_results, euclidean_results = [], []

In [ ]:
cosine_threshold = 0.9
euclidean_threshold = 15

#'''
for i in xrange(len(euclidean_results)):
    if euclidean_results[i] < euclidean_threshold:
        print sorted_vecor[i][0], sorted_vecor[i][1]['title_string']
#'''

'''        
for i in xrange(len(cosine_results)):
    if cosine_results[i] > cosine_threshold:
        print cosine_results[i]
        print sorted_vecor[i][0], sorted_vecor[i][1]['title_string']
'''

#### use the `key` to loop through all the titles

In [ ]:
results = []
#selected_key = 'http://www.cbssports.com/nba/news/nba-draft-lavar-ball-guarantees-lonzo-will-lead-lakers-to-playoffs-next-season/'
print selected_key
print '\n \n'
for key in vector_dict.keys():
    if key == selected_key:
        continue
    #vector_index = 0
    #cosine_result = cosine_similarity(vector_dict[selected_key]['vectors'][vector_index], vector_dict[key]['vectors'][vector_index])
    #euclidean_result = euclidean_distance(vector_dict[selected_key]['vectors'][vector_index], vector_dict[key]['vectors'][vector_index])
    #result = cosine_result
    start_time = time.time()
    cosine_result = cosine_similarity(vector_dict[selected_key]['vectors'], vector_dict[key]['vectors'])
    euclidean_result = euclidean_distance(vector_dict[selected_key]['vectors'], vector_dict[key]['vectors'])
    result = cosine_result
    print 'all the process takes {:.2f} seconds...'.format(time.time() - start_time)
    #result = euclidean_result
    #result = 10 * (1 - cosine_result) * euclidean_result
    #result = manhattan_distance(vector_dict[selected_key]['vectors'][0], vector_dict[key]['vectors'][0])
    #if result < 18:
    #if result > 0.92:
        #print key, result
        #print vector_dict[key]['pageView']
    
    #print cosine_result, euclidean_result, result
    results.append(result)

In [ ]:
plt.hist(results, bins=50)